# Scraping from OLX (Only inspected cars)

In [ ]:
import scrapy
import warnings
warnings.filterwarnings('ignore')
from scrapy.crawler import CrawlerProcess
import json
import csv
import time

class Olx(scrapy.Spider):
    
    name='olx'
    
    url='https://www.olx.in/api/relevance/v2/search?category=84&facet_limit=100&inspected=true&lang=en-IN&location=1000001&location_facet_limit=20&user=17d52ce2d67x283a1bea'
    headers={
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'
        }
    
    def __init__(self):
        with open('olx_scraped.csv', 'w') as csv_file:
            csv_file.write('car_type,location,brand,model,version,year,fuel_type,transmission_type,distance_driven,price\n')
    
    def start_requests(self):
        for page in range(0,25):
            yield scrapy.Request(url=self.url + '&page='+str(page), headers= self.headers, callback=self.parse)
            time.sleep(5)
            
            
            
    def parse(self, response):
        details=response.text
        data=json.loads(details)
        
        for info in data['data']:
            items={
                'car_body':info['car_body_type'],
                'location':info['locations_resolved']['ADMIN_LEVEL_1_name'],
                'Brand':info['parameters'][0]['formatted_value'],
                'Model':info['parameters'][1]['formatted_value'],
                'Variant':info['parameters'][2]['formatted_value'],
                'year':info['parameters'][3]['formatted_value'],
                'fuel':info['parameters'][4]['formatted_value'],
                'Transmission':info['parameters'][5]['formatted_value'],
                'Distance':info['parameters'][6]['value'],
                'Price':info['price']['value']['raw']
            }
            
            with open('olx_scraped.csv','a',newline='') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=items.keys())
                writer.writerow(items)

        
#Run scripts
process = CrawlerProcess()
process.crawl(Olx)
process.start()